# node structure 

Ideas: 
1-use the category of the meal as key of the dictionary of the dataset
2- this will be used in coordination with state.meal
3- the dataset will have the following form 
	{
		category : {{object},{object2},{object3}}
	}

# Problems: 
1- we have to check for daily calories so that they're satisfied which translates to checking the calories every three levels which represents a day 

In [ ]:
import queue
import math

from copy import deepcopy
from math import sin, cos, radians, atan2, sqrt

# here for the convenience: we chose to add a meal and a day data member.
# using this we can make the check for the daily constraints easier
# for instance for calories we'll just check if the meal is a dinner so that we know that we are at the end of the day and o check if the calories are ok for the day. which we will call a valid state: a valid state is a state that will be considered a worthy state to be expanded. if a state is not valid; we simply won't expand it. ( pruning)


# ------------------------------
# 1. Node Class Definition
# ------------------------------

class Node:

	def __init__(self, state, parent = None , meal='breakfast' , cost = 0 , day = 0, g=0, f=0):

		self.state= state        # Current meal plan state (7x3 matrix)
		self.day  = day
		self.cost = cost
		self.meal = meal           # Meal object (if applicable)this will signify what the current category of the state is, will be passed from parent to childre: parent: breakfast -> child: lunch 
								   # starts by the breakfast as default

		self.depth = 0 if parent is None else parent.depth + 1
		self.g = g                # Cumulative cost from start to this node
		self.f = f                # Evaluation cost (g + heuristic if applicable)

	def __hash__(self):
		if isinstance(self.state, list):
			state_tuple = tuple([tuple(row) for row in self.state])
			return hash(state_tuple)
		return hash(self.state)

	def __eq__(self, other):
		return isinstance(other, Node) and self.state == other.state

	def __gt__(self, other):
		return isinstance(other, Node) and self.f > other.f
	
	def __str__(self):
		
		for row in self.state:
			temp = list()
			for meal in row:
				print(meal['title'],end=10*' ')
			print()
		return ''


# class definition

In [5]:
import json

with open("dummy_meals.json", "r") as f:
	Dataset = json.load(f)

# ------------------------------
# 2. mealPlanning Class for Food Recommendation System
# ------------------------------

meal_to_num = {
	'breakfast' : 0 ,
	'lunch' : 1 ,
	'dinner': 2
}
num_to_meal = [ 'breakfast' , 'lunch' , 'dinner' ]


DAYS_IN_WEEK  = 7
MEELS_PER_DAY = 3

class mealPlanning:

	def __init__(self, initial_state, goal_state,allergies,dietType):

		self.initial_state = initial_state  # Empty meal plan (7x3 matrix)
		self.goal_state = goal_state        # Target meal plan with calorie and cost goals
		self.allergies  = allergies         # List of allergies (if any)
		self.diets = dietType

		# Constraints
		self.max_repetitions = 1  # Max times a dish can be repeated in a week

	def is_goal(self, current_state):
		#compatible with hill climbing 

		# note :
		# -----------------------------------
		# hard constraints considered here 
		#
		#  ( margin of error is considered )
		# a full grid 
		# cost of state < budget               
		# calories daily match for user input 
		# allergies + diet_type
		# ------------------------------------

		# we must check for the cost because it is global, also we must check if the grid is full
		full = all ( x is not None for row in current_state for x in row)
		if not full:
			return False
		
		# check cost + calories 
		calories_per_day , total_cost = self._calculate_plan_stats(current_state)

		if total_cost > self.goal_state['cost'] + self.goal_state['cost']*self.goal_state['cost_margin']:
			return False
		
		daily_calorie_target = self.goal_state['cal']
		if any( abs( x-daily_calorie_target ) >  self.goal_state['calories_margin'] * daily_calorie_target for x in calories_per_day ):
			return False
					
		# check allergies+diet_type conditions if met 
		for row in current_state:
			for meal in row:		
				if not ( all( allergy in meal['allergies'] for allergy in self.allergies) and all( diet in meal['diet_type'] for diet in self.diets) ):
					return False
		
		return True

	def _calculate_plan_stats(self, state):

		# return cost of the plan + array of calories for the days
		# used in goal checkin ( helper for goal checker )

		calories_per_day = list() # array of 7
		total_cost = 0
		
		for day in range(DAYS_IN_WEEK):

			temp_calories = 0 # calories for the current day 
			for meal in range(MEELS_PER_DAY):

				dish = state[day][meal]
				if dish is not None:
					temp_calories += dish['calories']
					total_cost += dish['cost']
				calories_per_day.append(temp_calories)
					
		return calories_per_day , total_cost

	def get_valid_actions(self, current_node):

		# we can use sets for optimization, then just do the difference
		# find the category of the meal (i.e breakfast ..Etc)

		# checking only hard constraints as mentioned in goal check function
		meal_idx = meal_to_num[current_node.meal]
		valid_meals = []
		
		for meal in Dataset[current_node.meal]:

			# check for cost 
			cost_condition = self.goal_state['cost'] + self.goal_state['cost']*self.goal_state['cost_margin']
			valid_cost = meal['cost'] + current_node.cost <= cost_condition
			if not valid_cost:
				continue
			
			# check how many times this meal has been repeted 
			count_repetitions = 0
			for i in range( 0 , current_node.day ):
				if meal['title'] == current_node.state[i][meal_idx]['title']:
					count_repetitions+=1
			if count_repetitions >= self.max_repetitions:
				continue
			
			# check allergies and diet_type conditions
			if not ( all( allergy in meal['allergies'] for allergy in self.allergies) and all( diet in meal['diet_type'] for diet in self.diets) ):
				continue
		
			# if it is dinner check for the day condition for calories 
			# daily_goal <<< or <<< daily_goal will be rejected
			if meal_idx == 2 and not self.is_valid_meal(meal, current_node.state[current_node.day]) :
				continue

			# passed all test => valid meal
			valid_meals.append(meal)

		return valid_meals
	
	def is_valid_meal(self, meal, DayMeals):
		
		# Check if the meal is valid based on nutritional balance
		# Replace this with your own nutritional balance check logic

		calories = sum(meal['calories'] for meal in DayMeals if meal is not None) + meal['calories']

		daily_calorie_target = self.goal_state['cal']
		calorie_margin = daily_calorie_target * self.goal_state['calories_margin']

		calories_valid = abs(calories - daily_calorie_target) <= calorie_margin

		# fats and other nutritionals maybe considered soft uncritical constraints 
		# should be imeplemented in the cost and heuristic 
		'''
			proteins = sum(meal['proteins'] for meal in DayMeals if meal is not None) + meal['proteins']
			fats     = sum(meal['fats'] for meal in DayMeals if meal is not None) + meal['fats']
			carbs    = sum(meal['carbs'] for meal in DayMeals if meal is not None) + meal['carbs']
			rating   = (sum(meal['rating'] for meal in DayMeals if meal is not None) + meal['rating'] ) /3
			daily_protein_target = self.goal_state.get('prot', 75)
			daily_fats_target    = self.goal_state.get('fat', 70)
			daily_carbs_target   = self.goal_state.get('carb', 150)
			avg_rating 			 = self.goal_state.get('rating', 3)

			protein_margin = daily_protein_target * self.goal_state['proteins_margin']
			fat_margin     = daily_fats_target    * self.goal_state['fats_margin']
			carb_margin    = daily_carbs_target   * self.goal_state['carbs_margin']
			avg_rating     = self.goal_state['avg_rating']

			protein_valid  = abs(proteins - daily_protein_target) <= protein_margin
			fat_valid      = abs(fats - daily_fats_target) <= fat_margin
			carb_valid     = abs(carbs - daily_carbs_target) <= carb_margin
			del_valid 	   = rating >= avg_rating

			return calories_valid and protein_valid and fat_valid and carb_valid and del_valid
		'''

		return calories_valid
			
	def expand_node(self, node, A_search=False , ucs_search=False):
		state = node.state

		# Find valid actions for this state
		valid_meals = self.get_valid_actions(node)

		# meal type
		day_idx  = node.day
		child_nodes = []

		# the grid is full
		if ( day_idx >= DAYS_IN_WEEK ):
			return []

		# breakfast : 0 , lunch : 1 , dinner : 2  
		meal_idx = meal_to_num[node.meal]
		for meal in valid_meals:

			child_state =  deepcopy(state)
			child_state[day_idx][meal_idx] = meal

			# adding dinner => move to the next day
			new_day   = node.day + 1 if meal_idx == 2 else node.day
			# meal_indx + 1 map to keywords breakfast lunch dinner  
			next_meal = num_to_meal[ ( meal_idx+1)%3 ]

			child_g , child_f = 0 , 0

			if ucs_search :
				child_g = self.ucs_search_meal_cost(meal,meal_idx)
				child_f = child_g 

			elif A_search :
				# this is calling path cost of A* + heuristic
				pass

			# else ...
			# will calculate nothing since this is DFS

			child = Node( state=child_state , parent=node , meal=next_meal , day=new_day , cost=node.cost+meal['cost'] , f=child_f , g=child_g )
			child_nodes.append(child)
			
			
			# for the ones responsible for UCS and A* to figure out    
		return child_nodes
	


	def ucs_search_meal_cost(self, dish , meal_indx):
		
		# NOTE the values here arent tested yet ( possible changes after testing )
		# ------------------------------------------------------------------------

		# if the goal is to have 100 protein in the day 
		# breakfast should be 25grams , lunch 40grams , dinner 35grams
		MEAL_TO_IDEAL_PERCENTAGE = [ 0.25 , 0.40 , 0.35 ]
		# dummy values for the global wheights for UCS path cost function
		WEIGHT_OF_COST = 0.4
		WEIGHT_OF_NUTRITIONS = 0.8
		WEIGHT_OF_DELICIOUSNESS = 0.01
		# --------------------------------------------------------------------

		# 100 protein in day => for breakfast 100 proteins * NutriontsIdeal
		NutriontsIdealFactor  = MEAL_TO_IDEAL_PERCENTAGE[ meal_indx ]

		protein_deficit  = abs ( dish['proteins']-  self.goal_state.get('prot', 75)  * NutriontsIdealFactor ) / ( self.goal_state.get('prot', 75)  * NutriontsIdealFactor )
		fat_deficit      = abs ( dish['fats']    -  self.goal_state.get('fats', 75)  * NutriontsIdealFactor ) / ( self.goal_state.get('fats', 75)  * NutriontsIdealFactor )
		carbs_deficit    = abs ( dish['carbs']   -  self.goal_state.get('carbs', 75) * NutriontsIdealFactor ) / ( self.goal_state.get('carbs', 75) * NutriontsIdealFactor )

		# this one is debatable 👀👀
		calories_deficit = abs ( dish['calories'] - self.goal_state.get('cal', 75)   * NutriontsIdealFactor ) / ( self.goal_state.get('cal', 75)   * NutriontsIdealFactor )

		# -------------
		#	get the importance form the slider 
		#   proteins > carbs > fat ( example )
		# -------------

		Protein_w = 0.2
		Fat_w     = 0.3 
		Carbs_w   = 0.4
		Calories_w= 0.2
				
		return dish["cost"] * WEIGHT_OF_COST + ( protein_deficit*Protein_w + fat_deficit*Fat_w + Carbs_w*carbs_deficit + calories_deficit*Calories_w ) * WEIGHT_OF_NUTRITIONS + ( 1 - dish['rating']/5 ) * WEIGHT_OF_DELICIOUSNESS

	def get_crow_flies_distance(self, coordinates, goal_coordinates):
		# For food recommendation, this could be nutritional distance
		# More sophisticated implementations might use a multi-dimensional distance metric
		
		# Using Euclidean distance for simplicity
		squared_diff_sum = 0
		for i in range(len(coordinates)):
			squared_diff_sum += (coordinates[i] - goal_coordinates[i]) ** 2
			
		return math.sqrt(squared_diff_sum)

	
	def print_node(self, node):
		if node.action is not None:
			print("Action:", node.action, "| Depth:", node.depth)
			dish_info = self.state_transition_model[node.action]
			print(f"Added dish: {dish_info['name']}, Calories: {dish_info['calories']}, Cost: {dish_info['cost']}")
		
		# Print the meal plan
		if node.state is not None:
			print("\nCurrent Meal Plan:")
			days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
			meals = ["Breakfast", "Lunch", "Dinner"]
			
			for day_idx, day in enumerate(days):
				print(f"\n{day}:")
				for meal_idx, meal in enumerate(meals):
					dish_id = node.state[day_idx][meal_idx]
					if dish_id is not None:
						dish_info = self.state_transition_model[dish_id]
						print(f"  {meal}: {dish_info['name']} ({dish_info['calories']} cal, ${dish_info['cost']})")
					else:
						print(f"  {meal}: ---")
'''
def is_valid_meal(self, meal, DayMeals):
	# Calculate total nutrients for the day with the new meal
	calories = sum(meal['calories'] for meal in DayMeals if meal is not None) + meal['calories']
	proteins = sum(meal['proteins'] for meal in DayMeals if meal is not None) + meal['proteins']
	fats = sum(meal['fats'] for meal in DayMeals if meal is not None) + meal['fats']
	carbs = sum(meal['carbs'] for meal in DayMeals if meal is not None) + meal['carbs']
	
	# Define reasonable ranges for daily nutrition
	# These could be user inputs or calculated based on goal state
	daily_calorie_target = self.goal_state.get('daily_calories', 2000)  # Default 2000 calories
	
	# Calculate acceptable ranges (typically 10-20% deviation is acceptable)
	calorie_margin = daily_calorie_target * 0.15  # 15% margin
	
	# Macro distribution targets (percentages of total calories)
	protein_target_pct = self.goal_state.get('protein_pct', 0.25)  # 25% of calories from protein
	fat_target_pct = self.goal_state.get('fat_pct', 0.30)  # 30% of calories from fat
	carb_target_pct = self.goal_state.get('carb_pct', 0.45)  # 45% of calories from carbs
	
	# Convert percentages to grams
	# Protein and carbs: 4 calories per gram, fat: 9 calories per gram
	protein_target_g = (daily_calorie_target * protein_target_pct) / 4
	fat_target_g = (daily_calorie_target * fat_target_pct) / 9
	carb_target_g = (daily_calorie_target * carb_target_pct) / 4
	
	# Define acceptable margins for macronutrients (20% deviation)
	protein_margin = protein_target_g * 0.20
	fat_margin = fat_target_g * 0.20
	carb_margin = carb_target_g * 0.20
	
	# Check if values are within acceptable ranges
	calories_valid = abs(calories - daily_calorie_target) <= calorie_margin
	protein_valid = abs(proteins - protein_target_g) <= protein_margin
	fat_valid = abs(fats - fat_target_g) <= fat_margin
	carb_valid = abs(carbs - carb_target_g) <= carb_margin
	
	# If we're checking dinner (the last meal of the day), be strict about totals
	# For breakfast and lunch, we should be more lenient since other meals will follow
	if len([m for m in DayMeals if m is not None]) == 2:  # This is dinner
		return calories_valid and protein_valid and fat_valid and carb_valid
	else:  # This is breakfast or lunch
		# For earlier meals, just make sure we're not exceeding daily targets
		return calories <= daily_calorie_target * 1.1 and proteins <= protein_target_g * 1.2

def init(self, initial_state, goal_state, allergies, dietType, 
			 calorie_margin_pct=0.15, protein_margin_pct=0.20, fat_margin_pct=0.20, carb_margin_pct=0.20):
	# Existing initialization code...
	self.calorie_margin_pct = calorie_margin_pct
	self.protein_margin_pct = protein_margin_pct 
	self.fat_margin_pct = fat_margin_pct
	self.carb_margin_pct = carb_margin_pct


def expand_node(self, node, use_cost=True, use_heuristic=False):
	# If the node represents a complete plan, don't expand further
	if node.day >= self.days_per_week:
		return []
	
	# Determine which meal type we're filling next
	meal_types = ["breakfast", "lunch", "dinner"]
	meal_type = meal_types[node.meal]
	
	# Get valid meals for this position
	valid_meals = self.get_valid_actions(node)
	
	# Generate child nodes
	children = []
	for meal in valid_meals:
		# Create a new state by copying the current state
		new_state = [row[:] for row in node.state]
		
		# Add the meal to the state
		new_state[node.day][node.meal] = meal
		
		# Determine the next position (day and meal)
		next_meal = (node.meal + 1) % 3
		next_day = node.day + 1 if next_meal == 0 else node.day
		
		# Calculate cost (could be financial cost, nutritional cost, etc.)
		# For now, using meal cost as the cost
		cost = node.cost + meal.cost
		
		# Calculate heuristic value if needed
		heuristic = 0
		if use_heuristic:
			# Remaining days * average daily cost as a simple heuristic
			remaining_days = self.days_per_week - next_day
			avg_daily_cost = self.goal_state.get('budget', 0) / self.days_per_week  
			heuristic = remaining_days * avg_daily_cost
			
			# Adjust heuristic based on how well we're meeting daily nutritional targets
			if node.meal == 2:  # After adding dinner, evaluate day completeness
				day_nutrients = self._calculate_day_nutrients(new_state[node.day])
				nutrient_balance = self._evaluate_nutrient_balance(day_nutrients)
				# Penalize days with poor nutritional balance
				heuristic += (1 - nutrient_balance) * 50  # Higher penalty for poor balance
		
		# Calculate f value (g + h)
		f_value = cost + heuristic if use_heuristic else cost
		
		# Create child node
		child = Node(
			state=new_state,
			day=next_day,
			meal=next_meal,
			cost=cost,
			f_value=f_value,
			parent=node,
			action=meal
		)
		
		children.append(child)
	
	return children



'''

# Example usage with a toy meal planning problem
def toy_test_meal_planning_problem():
	# Define a database of dishes with their properties
	
	# Define the initial state: empty 7x3 matrix (7 days, 3 meals per day)
	initial_state = [[None for _ in range(3)] for _ in range(7)]
	# to be taken from the user input
	
	# Define the goal state: target calories and cost for the week
	goal_state = {
		'cal': 8400,  # Average 1200 calories per day
		'calories_margin': 0.1,  # Acceptable margin of error
		'proteins_margin': 0.15,  # Acceptable margin of error
		'carbs_margin': 0.2,  # Acceptable margin of error
		'fats_margin': 0.1,  # Acceptable margin of error
		'cost': 140.00,  # Weekly food budget
		'cost_margin': 0.15,  # Acceptable margin of error
		'avg_rating': 3
		}
	
	# Create the problem instance
	meal_plan_problem = mealPlanning(
		initial_state=initial_state,
		goal_state=goal_state,
		allergies=[],
		dietType=[]
	)
	
	# Test the problem methods
	print("Testing meal planning problem:")
	
	# Test is_goal method
	print("\nTesting is_goal:")
	print("Is empty plan a goal?", meal_plan_problem.is_goal(initial_state))  # Should be False
	
	
	# Test expand_node
	print("\nTesting expand_node:")
	start_node = Node(state=initial_state, g=0)
	children = meal_plan_problem.expand_node(start_node)
	# print(f"Number of child nodes: {len(children)}")
	
	# # Test print_node
	# print("\nTesting print_node:")
	# meal_plan_problem.print_node(start_node)
	
	# if children:
		# print("\nSample child node:")
		# meal_plan_problem.print_node(children[0])
	

if __name__ == "__main__":
	print( "start dummy testing")
	# Run the toy test for meal planning
	toy_test_meal_planning_problem()

start dummy testing
Testing meal planning problem:

Testing is_goal:
Is empty plan a goal? False

Testing expand_node:


In [7]:
# ------------------------------
# 3. General Search Class
# ------------------------------
class GeneralSearch:
	def __init__(self, problem):

		self.problem = problem
		self.A_search = False
		self.ucs_search = False

	def set_frontier(self, search_strategy="depth_first"):

		if search_strategy == "depth_first":

			frontier = queue.LifoQueue()

		elif search_strategy == "uniform_cost":

			frontier = queue.PriorityQueue()
			self.ucs_search = True

		elif search_strategy == "A*":

			frontier = queue.PriorityQueue()
			self.A_search = True

		else:
			# If an unsupported search strategy is provided, raise an error to alert the user.
			raise ValueError("Unsupported search strategy: " + str(search_strategy))

		return frontier

	def search(self, search_strategy="breadth_first", max_depth=float('inf')):

		frontier = self.set_frontier(search_strategy)
		initial_node = Node(self.problem.initial_state)
		frontier.put(initial_node)

		while not frontier.empty():

			node = frontier.get()

			if self.problem.is_goal(node.state):
				return node
		

			if node.depth > max_depth:
				continue

			
			children = self.problem.expand_node(node,self.A_search,self.ucs_search)

			for child_node in children:
				frontier.put(child_node)


		return None  # Return None if no solution is found.

def toy_test_search():
	# Define a database of dishes with their properties
	
	# Define the initial state: empty 7x3 matrix (7 days, 3 meals per day)
	initial_state = [[None for _ in range(3)] for _ in range(7)]
	# to be taken from the user input
	
	# Define the goal state: target calories and cost for the week
	goal_state = {
		'cal': 1500,  # Average 1200 calories per day
		'calories_margin': 200,  # Acceptable margin of error
		'proteins_margin': 200,  # Acceptable margin of error
		'carbs_margin': 200,  # Acceptable margin of error
		'fats_margin': 200,  # Acceptable margin of error
		'cost': 400,  # Weekly food budget
		'cost_margin': 1,  # Acceptable margin of error
		'avg_rating': 4.0
	}
	
	# Create the problem instance
	meal_plan_problem = mealPlanning(
		initial_state=initial_state,
		goal_state=goal_state,
		allergies=["gluten-free"],
		dietType=[]
	)
	
	# Test the problem methods
	print("Testing search for meal planning problem:")
	
	searching_object = GeneralSearch(meal_plan_problem)
	mealPlanning.print_node(searching_object.search("depth_first"))
	

if __name__ == "__main__":
	print( "start dummy testing")
	# Run the toy test for meal planning
	toy_test_search()

start dummy testing
Testing search for meal planning problem:


TypeError: mealPlanning.print_node() missing 1 required positional argument: 'node'